In [1]:
import pandas as pd
import numpy as np

In [8]:
from datetime import datetime, timedelta

In [194]:
data = pd.read_csv("../data/processed/cut_off_price_with_weather_time.csv",parse_dates=["Trade_Datetime","ecmen_pulicate_time","ecmop_pulicate_time","gfsen_pulicate_time","gfsop_pulicate_time"])

In [44]:
data.head()

,Trade_Datetime,Contract_Delivery_Date,Low_Price,High_Price,Open_Price,Close_Price,Trade_Quantity,Vwap,ecmen_pulicate_time,ecmop_pulicate_time,gfsen_pulicate_time,gfsop_pulicate_time,dst_flag
0,2015-01-28 13:29:00,1503,2.841,2.846,2.843,2.846,4399,2.843,2015-01-28 01:40:00,2015-01-28 12:55:00,2015-01-28 12:55:00,2015-01-28 11:13:00,0
1,2015-01-28 13:30:00,1503,2.838,2.847,2.845,2.839,862,2.843,2015-01-28 01:40:00,2015-01-28 12:55:00,2015-01-28 12:55:00,2015-01-28 11:13:00,0
2,2015-01-28 13:31:00,1503,2.837,2.841,2.840,2.837,137,2.839,2015-01-28 01:40:00,2015-01-28 12:55:00,2015-01-28 12:55:00,2015-01-28 11:13:00,0
3,2015-01-28 13:32:00,1503,2.837,2.839,2.838,2.837,111,2.838,2015-01-28 01:40:00,2015-01-28 12:55:00,2015-01-28 12:55:00,2015-01-28 11:13:00,0
4,2015-01-28 13:33:00,1503,2.835,2.839,2.837,2.837,84,2.837,2015-01-28 01:40:00,2015-01-28 12:55:00,2015-01-28 12:55:00,2015-01-28 11:13:00,0


In [204]:
data = pd.read_csv("../data/processed/WeatherData/gfsen_weather_subclass.csv",parse_dates=['Trans_INIT_Time'])

In [205]:
data

,Trans_INIT_Time,VALUE_hdd,10Y_NORMAL_hdd,Sum_Value_hdd,Delta_Full_hdd,Delta_Sub_hdd,VALUE_cdd,10Y_NORMAL_cdd,Sum_Value_cdd,Delta_Full_cdd,Delta_Sub_cdd
0,2015-01-01 00:55:00,34.39,28.51,500.45,0.00,0.00,0.24,0.19,3.70,0.00,0.00
1,2015-01-01 06:55:00,34.19,28.51,493.01,-7.44,-3.42,0.24,0.19,3.50,-0.20,-0.11
2,2015-01-01 12:55:00,31.12,28.67,490.88,2.09,0.32,0.42,0.17,3.49,0.10,0.13
3,2015-01-01 18:56:00,31.31,28.67,502.42,11.54,13.98,0.42,0.17,3.43,-0.06,-0.12
4,2015-01-02 00:55:00,31.01,28.67,508.42,6.00,6.12,0.40,0.17,3.05,-0.38,-0.33
...,...,...,...,...,...,...,...,...,...,...,...
9123,2021-03-30 19:29:00,16.56,15.32,188.50,2.34,1.43,0.98,1.01,12.20,-0.60,-0.25
9124,2021-03-31 01:28:00,16.18,15.32,187.04,-1.46,-1.40,1.00,1.01,12.19,-0.01,-0.18
9125,2021-03-31 07:28:00,16.13,15.32,187.57,0.53,3.62,0.96,1.01,12.81,0.62,-0.06
9126,2021-03-31 13:28:00,20.84,15.06,182.02,-0.96,-1.31,0.39,1.02,12.61,-0.77,-0.42


In [206]:
data['Trans_INIT_Time'].dt.time.unique()

array([datetime.time(0, 55), datetime.time(6, 55), datetime.time(12, 55),
       datetime.time(18, 56), datetime.time(1, 28), datetime.time(7, 28),
       datetime.time(13, 28), datetime.time(19, 29)], dtype=object)

In [39]:
data.loc[data.dst_flag==1, 'Trade_Datetime'] = data[data["dst_flag"]==1]["Trade_Datetime"].apply(lambda x : x + timedelta(hours=-1))
data.loc[data.dst_flag==1, 'ecmen_pulicate_time'] = data[data["dst_flag"]==1]["ecmen_pulicate_time"].apply(lambda x : x + timedelta(hours=-1))

In [47]:
data.set_index('Trade_Datetime',inplace=True)

data.dropna(how='any',inplace=True)
time_point = []
st_time = datetime.strptime('6:00', '%H:%M').time()
et_time = datetime.strptime('16:00', '%H:%M').time()
time_point = [st_time, et_time]

timels = data['ecmen_pulicate_time'].dt.time.unique()
for item in timels:
    if (item >= st_time) & (item <= et_time):
        time_point.append(item)
time_point.sort()
time_point_pairs = list(zip(time_point, time_point[1:]))

In [90]:
time_point_pairs

[(datetime.time(6, 0), datetime.time(13, 40)),
 (datetime.time(13, 40), datetime.time(16, 0))]

In [98]:
first_peirod = data[['Contract_Delivery_Date', 'Vwap', 'ecmen_pulicate_time', 'dst_flag']].between_time(time_point_pairs[0][0], time_point_pairs[0][1], include_start=False)

In [153]:
first_peirod_grouped = first_peirod[['Contract_Delivery_Date',"Vwap"]].groupby(['Contract_Delivery_Date', pd.Grouper(freq='D')])

In [152]:
first_peirod["Normal_Vwap"]  = first_peirod_grouped.transform(lambda x : x/ x.values[0])

In [148]:
first_peirod

,Contract_Delivery_Date,Vwap,ecmen_pulicate_time,dst_flag,Normal_vwap,Normal_Vwap
Trade_Datetime,,,,,,
2015-01-28 13:29:00,1503,2.843,2015-01-28 01:40:00,0,1.000000,1.000000
2015-01-28 13:30:00,1503,2.843,2015-01-28 01:40:00,0,1.000000,1.000000
2015-01-28 13:31:00,1503,2.839,2015-01-28 01:40:00,0,0.998593,0.998593
2015-01-28 13:32:00,1503,2.838,2015-01-28 01:40:00,0,0.998241,0.998241
2015-01-28 13:33:00,1503,2.837,2015-01-28 01:40:00,0,0.997890,0.997890
...,...,...,...,...,...,...
2021-03-31 13:36:00,2105,2.612,2021-03-31 01:40:00,1,0.990520,0.990520
2021-03-31 13:37:00,2105,2.611,2021-03-31 01:40:00,1,0.990140,0.990140
2021-03-31 13:38:00,2105,2.611,2021-03-31 01:40:00,1,0.990140,0.990140


In [160]:
res = first_peirod.groupby(['Contract_Delivery_Date', pd.Grouper(freq='D')]).agg(Normal_Vwap=('Normal_Vwap', list),ecmen_pulicate_time=('ecmen_pulicate_time', 'first'),dst_flag=('dst_flag', 'first'))

In [161]:
res

Normal_Vwap  \
Contract_Delivery_Date Trade_Datetime                                                      
1503                   2015-01-28      [1.0, 1.0, 0.998593035525853, 0.99824129440731...   
                       2015-01-29      [1.0, 0.9989495798319329, 0.998249299719888, 0...   
                       2015-01-30      [1.0, 0.9992481203007518, 0.9981203007518795, ...   
                       2015-02-02      [1.0, 1.0003746721618583, 1.0003746721618583, ...   
                       2015-02-03      [1.0, 1.0007401924500372, 1.0007401924500372, ...   
...                                                                                  ...   
2104                   2021-03-26      [1.0, 0.9992205767731879, 0.9996102883865939, ...   
                       2021-03-29      [1.0, 1.0011741682974558, 1.0011741682974558, ...   
2105                   2021-03-29      [1.0, 0.9984922728986053, 0.9977384093479079, ...   
                       2021-03-30      [1.0, 0.99962602842184, 0.99925205684368, 0.99...   
                       2021-03-31      [1.0, 0.9992415623814941, 0.9996207811907472, ...   

                                      ecmen_pulicate_time  dst_flag  
Contract_Delivery_Date Trade_Datetime                                
1503                   2015-01-28     2015-01-28 01:40:00         0  
                       2015-01-29     2015-01-29 01:40:00         0  
                       2015-01-30     2015-01-30 01:40:00         0  
                       2015-02-02     2015-02-02 01:40:00         0  
                       2015-02-03     2015-02-03 01:40:00         0  
...                                                   ...       ...  
2104                   2021-03-26     2021-03-26 01:40:00         1  
                       2021-03-29     2021-03-29 01:40:00         1  
2105                   2021-03-29     2021-03-29 01:40:00         1  
                       2021-03-30     2021-03-30 01:40:00         1  
                       2021-03-31     2021-03-31 01:40:00         1  

[1620 rows x 3 columns]

In [165]:
first_peirod.index = first_peirod.index + pd.DateOffset(hours=-1)

In [166]:
first_peirod

,Contract_Delivery_Date,Vwap,ecmen_pulicate_time,dst_flag,Normal_vwap,Normal_Vwap
Trade_Datetime,,,,,,
2015-01-28 13:29:00,1503,2.843,2015-01-28 01:40:00,0,1.000000,1.000000
2015-01-28 13:30:00,1503,2.843,2015-01-28 01:40:00,0,1.000000,1.000000
2015-01-28 13:31:00,1503,2.839,2015-01-28 01:40:00,0,0.998593,0.998593
2015-01-28 13:32:00,1503,2.838,2015-01-28 01:40:00,0,0.998241,0.998241
2015-01-28 13:33:00,1503,2.837,2015-01-28 01:40:00,0,0.997890,0.997890
...,...,...,...,...,...,...
2021-03-31 13:36:00,2105,2.612,2021-03-31 01:40:00,1,0.990520,0.990520
2021-03-31 13:37:00,2105,2.611,2021-03-31 01:40:00,1,0.990140,0.990140
2021-03-31 13:38:00,2105,2.611,2021-03-31 01:40:00,1,0.990140,0.990140


In [193]:
first_peirod[first_peirod.dst_flag == 1].index

DatetimeIndex(['2015-03-09 06:00:00', '2015-03-09 06:01:00',
               '2015-03-09 06:02:00', '2015-03-09 06:03:00',
               '2015-03-09 06:04:00', '2015-03-09 06:05:00',
               '2015-03-09 06:06:00', '2015-03-09 06:07:00',
               '2015-03-09 06:08:00', '2015-03-09 06:09:00',
               ...
               '2021-03-31 13:31:00', '2021-03-31 13:32:00',
               '2021-03-31 13:33:00', '2021-03-31 13:34:00',
               '2021-03-31 13:35:00', '2021-03-31 13:36:00',
               '2021-03-31 13:37:00', '2021-03-31 13:38:00',
               '2021-03-31 13:39:00', '2021-03-31 13:40:00'],
              dtype='datetime64[ns]', name='Trade_Datetime', length=451894, freq=None)

In [181]:
first_peirod

,Contract_Delivery_Date,Vwap,ecmen_pulicate_time,dst_flag,Normal_vwap,Normal_Vwap
Trade_Datetime,,,,,,
2015-01-28 13:29:00,1503,2.843,2015-01-28 01:40:00,0,1.000000,1.000000
2015-01-28 13:30:00,1503,2.843,2015-01-28 01:40:00,0,1.000000,1.000000
2015-01-28 13:31:00,1503,2.839,2015-01-28 01:40:00,0,0.998593,0.998593
2015-01-28 13:32:00,1503,2.838,2015-01-28 01:40:00,0,0.998241,0.998241
2015-01-28 13:33:00,1503,2.837,2015-01-28 01:40:00,0,0.997890,0.997890
...,...,...,...,...,...,...
2021-03-31 13:36:00,2105,2.612,2021-03-31 01:40:00,1,0.990520,0.990520
2021-03-31 13:37:00,2105,2.611,2021-03-31 01:40:00,1,0.990140,0.990140
2021-03-31 13:38:00,2105,2.611,2021-03-31 01:40:00,1,0.990140,0.990140
